Copy files from subfolders to directory

In [6]:
import datetime as dt
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path
from glob import iglob


convert lowercase to upper

In [7]:
import os

path = r'/Users/baranyildirim/Downloads/Data_ibutton_05092022/'

for file in os.listdir(path):
    os.rename(path + file, path + file.upper())

    

Regroup files as Temp and RH

In [8]:
import os
import shutil

src = r'/Users/baranyildirim/Downloads/Data_ibutton_05092022/'
dest = r'/Users/baranyildirim/Downloads/RH/'

for path, subdirs, files in os.walk(src):
    for name in files:
        if name.startswith('H'):
            filename = os.path.join(path, name)
            shutil.copy2(filename, dest)

In [9]:
src = r'/Users/baranyildirim/Downloads/Data_ibutton_05092022/'
dest = r'/Users/baranyildirim/Downloads/Temp/'

for path, subdirs, files in os.walk(src):
    for name in files:
        if name.startswith('T'):
            filename = os.path.join(path, name)
            shutil.copy2(filename, dest)

create clean version of all csv files

In [18]:
import datetime as dt
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path
from glob import iglob



path = r'/Users/baranyildirim/Downloads/RH' # use your path
all_files = glob.glob(path + "/*.CSV")

cnt = 0
finalexcelsheet = pd.DataFrame()
for filename in all_files:
    df = pd.read_csv(filename, sep='§', index_col=None, header=0, engine='python', dayfirst=True)
    s1 = filename
    s2 = "H_"
    s= s1[s1.index(s2) + len(s2):]
    split_string = s.split("_")
    household = split_string[0]

    df = df.iloc[18:]
    df = df.rename(columns={ df.columns[0]: household })
    df = df[household].str.split(",")
    df = df.rename(filename)
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    df2 = pd.DataFrame(df)
    split_df1 = pd.DataFrame(df2[filename].tolist())

    
    #split_df1 = split_df1[split_df1[0] != '31/09/21 23:05:01']
    #split_df1 = split_df1[split_df1[0] != '31/09/21 23:20:01']
    #split_df1 = split_df1[split_df1[0] != '31/09/21 23:35:01']
    #split_df1 = split_df1[split_df1[0] != '31/09/21 23:50:01']
    
    
    #
    #split_df1['Dates'] = pd.to_datetime(split_df1[0], dayfirst=True).dt.date
    #split_df1['Time'] = pd.to_datetime(split_df1[0]).dt.time
    
    split_df1[household] = split_df1[[2, 3]].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)
    split_df1 = split_df1.drop([2, 3], axis=1, errors='ignore')
    split_df1[household] = split_df1[household].apply(lambda x: x.replace(',','.'))
    split_df1[household] = split_df1[household].astype('float64')
    split_df1[1] = split_df1[1].astype('string')
    for index, row in split_df1.iterrows():
        if split_df1[1][index] == 'F':
            split_df1[household][index] = (split_df1[household][index]-32)*(5/9)
            split_df1.loc[index, 1] = 'C'
    
    
    split_df1.rename(columns = {0: 'DateTime'}, inplace = True)
    
    split_df1 = split_df1.drop([1], axis=1, errors='ignore')
    
    
    split_df1['DateTime'] = pd.to_datetime(split_df1['DateTime']).dt.strftime("%d/%m/%Y %H:%M:%S")
    
    split_df1['DateTime'] = split_df1['DateTime'].astype('datetime64')
    split_df1['DateTime'] = split_df1['DateTime'].agg(lambda x : x.round('15min'))
    
    #split_df1 = split_df1.set_index('DateTime')
    split_df1 = split_df1.dropna()
    
    split_df1 = split_df1.sort_values(['DateTime'])
    


    finalexcelsheet = split_df1
    finalexcelsheet = finalexcelsheet.drop_duplicates()
    cnt +=1
    from pathlib import Path
    p = Path(r'/Users/baranyildirim/Downloads/RH/cleansed_csvs/')
    finalexcelsheet.to_csv(Path(p, 'H_' + str(household)+ '_' + str(cnt) + '.csv'), index=False)
    
    
    
    

    print(split_df1.memory_usage(index=False, deep=True))
    print(split_df1.info())
    
split_df1

DateTime    13848
0044AA      13848
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1731 entries, 0 to 1730
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DateTime  1731 non-null   datetime64[ns]
 1   0044AA    1731 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.6 KB
None
DateTime    13848
0049AA      13848
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1731 entries, 0 to 1730
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DateTime  1731 non-null   datetime64[ns]
 1   0049AA    1731 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.6 KB
None
DateTime    13872
084AA       13872
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1734 entries, 0 to 1733
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 

,DateTime,0088AA
0,2022-08-18 13:30:00,70.701
1,2022-08-18 13:45:00,52.614
2,2022-08-18 14:00:00,53.878
3,2022-08-18 14:15:00,53.950
4,2022-08-18 14:30:00,52.050
...,...,...
1724,2022-09-05 12:30:00,51.709
1725,2022-09-05 12:45:00,50.580
1726,2022-09-05 13:00:00,51.222
1727,2022-09-05 13:15:00,49.291


getting householdIDs from csv files in given folder

In [19]:
path = r'/Users/baranyildirim/Downloads/RH/cleansed_csvs'
all_files = glob.glob(path + "/*.csv")

list_hh = []

cnt = 0
finalexcelsheet = pd.DataFrame()
for filename in all_files:
    df = pd.read_csv(filename, sep='comma', index_col=None, header=0, engine='python')
    s1 = filename
    s2 = "H_"
    s= s1[s1.index(s2) + len(s2):]
    split_string = s.split("_")
    household = split_string[0]
    
    list_hh.append(str(household))
    

Count the appearance and store them in a dictionary

In [20]:
dict_hh = {i:list_hh.count(i) for i in list_hh}


Checking the number of appeariance

In [21]:
diff_dict_hh = dict((k, v) for k, v in dict_hh.items() if v < 2)
key_list = list(diff_dict_hh.keys())
print(key_list)

['0085AA', '0088AA', '084AA', '0044AA', '0076AA', '0049AA', '0007AA', '0064AA']


In [23]:
diff_dict_hh = dict((k, v) for k, v in dict_hh.items() if v > 2) #write the controlling number of appereance
key_list = list(diff_dict_hh.keys())
print(key_list)

[]


Checking the difference

In [24]:
list_hh = list(set(list_hh))

print(len(list_hh))

8


In [25]:
print(len(dict_hh))
print(sum(dict_hh.values()))

8
8


Comparison with serial numbers excel

In [27]:
list_ibutton = pd.read_excel(r'/Users/baranyildirim/Downloads/iButton_serial_numbers.xlsx', index_col=0)
list_serial = list_ibutton['Unnamed: 2'].values.tolist()
diff_list = [item for item in list_hh if item not in list_serial]
#diff_list = pd.DataFrame({'col':diff_list})
diff_list = list(set(diff_list))
diff_list

['0044AA', '0049AA', '0064AA', '0085AA', '0076AA', '0007AA', '084AA', '0088AA']

In [31]:
diff_list1 = [item for item in list_serial if item not in list_hh]
#diff_list = pd.DataFrame({'col':diff_list})
diff_list1

['ID Menage',
 '044000005AA',
 '044000019AB',
 '044000073AA',
 '044000102AA',
 '044000133AA',
 '044000137AA',
 '044000140AA',
 '044000189AA',
 '044000213AA',
 '044000218AA',
 '044000241AA',
 '044000309AA',
 '044000311AA',
 '044000312AA',
 '044000317AA',
 '044000340AC',
 '044000350AA',
 '044000365AA',
 '044000389AA',
 '044000391AA',
 '044000407AA',
 '044000408AA',
 '044000419AA',
 '044000421AA',
 '041000010AA',
 '041000015AA',
 '041000081AA',
 '041000095AA',
 '041000102AA',
 '041000111AA',
 '041000143AA',
 '041000150AA',
 '041000158AA',
 '041000191AA',
 '041000222AA',
 '041000235AA',
 '041000252AA',
 '041000293AA',
 '041000298AA',
 '041000300AA',
 '041000321AA',
 '041000336AA',
 '041000338AA',
 '041000348AA',
 '041000353AA',
 '041000364AA',
 '041000389AA',
 '041000423AA',
 '001000001AA',
 '001000004AB',
 '001000005AC',
 '001000011AA',
 '001000030AB',
 '001000089AB',
 '001000092AB',
 '001000096AA',
 '001000098AA',
 '001000100AA',
 '001000105AA',
 '001000111AA',
 '001000112AA',
 '00100011

Merging csv files belong to same householdID

In [32]:
import glob
import os
import pandas

def create_merged_csv(key, filelist):
    outfile = 'Concat_{}.csv'.format(key)
    l = []
    for filename in filelist:
        df = pandas.read_csv(filename)
        l.append(df)
    pd.concat(l).to_csv(outfile, index=False)

def find_filesets(path='.'):
    csv_files = {}
    for name in glob.glob("{}/*_*.csv".format(path)):
        key = '_'.join(name.split('_')[2:-1])
        csv_files.setdefault(key, []).append(name)
        
       
    for key,filelist in csv_files.items(): 
        print(key)
        create_merged_csv(key, filelist)

TEST_DIR_NAME=r'/Users/baranyildirim/Downloads/RH/cleansed_csvs'
os.chdir(r'/Users/baranyildirim/Downloads/RH/merged_csvs')
find_filesets(TEST_DIR_NAME)

0085AA
0088AA
084AA
0044AA
0076AA
0049AA
0007AA
0064AA


Merging cleansed household data into one table

In [34]:
path = r'/Users/baranyildirim/Downloads/RH/merged_csvs/folder' # use your path
all_files = glob.glob(path + "/*.csv")

for filename in all_files:
    #df = pd.read_csv(filename, sep=',', index_col=None, header=None, engine='python')
    df = pd.read_csv(filename, sep=',', index_col=None, header=0, engine='python')
    df = df.groupby('DateTime').mean().reset_index()
    df = df.drop_duplicates(subset='DateTime', keep='first')
    
    df['DateTime'] = df['DateTime'].astype('datetime64')
    df = df.sort_values(['DateTime'],ascending=False)
    df = df.set_index('DateTime')
    
    
path = r'/Users/baranyildirim/Downloads/RH/merged_csvs' # use your path
all_files = glob.glob(path + "/*.csv")

for filename in all_files:
    df1 = pd.read_csv(filename, sep=',', index_col=None, header=None, engine='python')
    s1 = filename
    s2 = "Concat_"
    s= s1[s1.index(s2) + len(s2):]
    split_string = s.split("_")
    household = split_string[0]
    household = household.rsplit('.', 1)[0]
    if 2 in df1.columns:
    
        df1 = df1.iloc[1:]
        df1.rename(columns = {1: 'first', 2:'second'}, inplace = True)
        
        df1['first'] = df1['first'].replace('nan', np.nan).fillna(0)
        df1['first'] = df1['first'].astype('float64')
        df1['second'] = df1['second'].replace('nan', np.nan).fillna(0)
        df1['second'] = df1['second'].astype('float64')
        #df['first'] = df['first'].apply(lambda x: x.replace(',','.'))
        #df['second'] = df['second'].apply(lambda x: x.replace(',','.'))
        df1[household]= df1['first'] +df1['second']
        
        df1.drop('first', axis=1, inplace=True) 
        df1.drop('second', axis=1, inplace=True) 
        
        df1.rename(columns = {0: 'DateTime'}, inplace = True)
        df1 = df1.groupby('DateTime').mean().reset_index()
        df1 = df1.drop_duplicates(subset='DateTime', keep='first')
        df1['DateTime'] = df1['DateTime'].astype('datetime64')
        df1 = df1.sort_values(['DateTime'],ascending=False)
        df1 = df1.set_index('DateTime')

        df1.info()
        
        
        df =df1.join(df, lsuffix="_left", rsuffix="_right", how='outer')
    else:
        
        df1 = pd.read_csv(filename, sep=',', index_col=None, header=None, engine='python')
        s1 = filename
        s2 = "Concat_"
        s= s1[s1.index(s2) + len(s2):]
        split_string = s.split("_")
        household = split_string[0]
        household = household.rsplit('.', 1)[0]
       
        df1 = df1.iloc[1:]
        
        df1.rename(columns = {1: household}, inplace = True)
        df1[household] = df1[household].replace('nan', np.nan).fillna(0)
        df1[household] = df1[household].astype('float64')
        
        
        df1.rename(columns = {0: 'DateTime'}, inplace = True)
        df1 = df1.groupby('DateTime').mean().reset_index()
        df1 = df1.drop_duplicates(subset='DateTime', keep='first')
        df1['DateTime'] = df1['DateTime'].astype('datetime64')
        df1 = df1.sort_values(['DateTime'],ascending=False)
        df1 = df1.set_index('DateTime')
        
        df1.info()
        df =df1.join(df, how='outer')
        
        
        
df = df.sort_values(['DateTime'],ascending=False)

       
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1731 entries, 2022-09-05 12:30:00 to 2022-08-18 12:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0076AA  1731 non-null   float64
dtypes: float64(1)
memory usage: 27.0 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1731 entries, 2022-09-05 12:45:00 to 2022-08-18 12:15:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0049AA  1731 non-null   float64
dtypes: float64(1)
memory usage: 27.0 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1731 entries, 2022-09-05 12:30:00 to 2022-08-18 12:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0044AA  1731 non-null   float64
dtypes: float64(1)
memory usage: 27.0 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1652 entries, 2022-09-05 13:15:00 to 2022-08-19 08:30:00
Data col

,084AA,0085AA,0007AA,0064AA,0044AA,0049AA,0076AA,0088AA
DateTime,,,,,,,,
2022-09-05 14:15:00,72.834,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-05 14:00:00,66.533,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-05 13:45:00,68.217,66.406,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-05 13:30:00,67.633,66.406,NaN,NaN,NaN,NaN,NaN,51.862
2022-09-05 13:15:00,66.977,65.267,73.671,61.502,NaN,NaN,NaN,49.291
...,...,...,...,...,...,...,...,...
2022-08-18 13:00:00,68.873,NaN,71.645,NaN,64.547,57.439,75.449,NaN
2022-08-18 12:45:00,NaN,NaN,70.486,NaN,65.077,56.891,74.885,NaN
2022-08-18 12:30:00,NaN,NaN,79.183,NaN,66.202,59.222,76.506,NaN


In [35]:
cols=df.columns.tolist()
cols.sort(reverse=True)
df=df[cols]

Reseting DateTime as an index

In [175]:
df = df.reset_index('DateTime')

In [67]:
df = df.replace(to_replace=0, value=np.nan)

In case of need for a filter

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1738 entries, 2022-09-05 14:15:00 to 2022-08-18 12:00:00
Freq: -15T
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   084AA   1734 non-null   float64
 1   0088AA  1729 non-null   float64
 2   0085AA  1731 non-null   float64
 3   0076AA  1731 non-null   float64
 4   0064AA  1652 non-null   float64
 5   0049AA  1731 non-null   float64
 6   0044AA  1731 non-null   float64
 7   0007AA  1732 non-null   float64
dtypes: float64(8)
memory usage: 122.2 KB


In case of filterin for date and time run the cell below

In [209]:
filtered_df = df.loc[(df['DateTime'] >= '2022-01-01 00:00:00')
                     & (df['DateTime'] < '2022-02-01 00:00:00')]
filtered_df

,DateTime,077000423AA,056000216AA,056000210AA,056000157AA,056000151AA,056000132AA,056000111AA,056000108AB,056000103AC,...,001000100AA,001000098AA,001000096AA,001000092AB,001000089AB,001000030AB,001000011AA,001000005AC,001000004AB,001000001AA
8510,2022-01-31 23:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.867,25.260,15.032,27.921,19.425,22.204,21.215,21.343,23.228,21.478
8511,2022-01-31 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.231,25.952,15.741,25.881,18.722,21.484,21.215,20.627,21.832,21.478
8512,2022-01-31 23:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.550,24.567,14.321,25.881,18.018,20.762,21.215,21.343,21.132,21.478
8513,2022-01-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.550,24.567,15.032,25.881,17.312,20.038,20.520,19.910,21.832,21.478
8514,2022-01-31 22:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.867,25.260,15.032,24.513,18.018,19.310,19.124,18.469,19.728,21.478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11481,2022-01-01 01:00:00,NaN,20.107,20.545,27.806,14.114,24.671,23.974,30.343,NaN,...,29.867,32.109,19.267,35.941,27.054,35.395,29.450,31.177,37.515,27.687
11482,2022-01-01 00:45:00,NaN,20.107,19.823,27.806,13.386,23.280,23.276,31.025,NaN,...,29.867,31.431,19.267,35.941,25.680,35.395,29.450,29.099,36.850,28.370
11483,2022-01-01 00:30:00,NaN,20.846,19.823,27.104,13.386,23.280,23.276,31.025,NaN,...,30.550,31.431,18.564,35.941,24.991,35.395,28.772,29.099,36.850,29.051
11484,2022-01-01 00:15:00,NaN,20.846,20.545,27.806,14.840,22.583,23.276,31.705,NaN,...,29.867,32.785,18.564,35.941,24.991,33.373,28.772,29.099,36.850,27.687


Saving the csv file

In [177]:
df.to_csv(r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Update Files\Pilot_RH.csv', index = False)

SQL script

In [37]:

df_col = pd.DataFrame(df.columns, columns=['cols'])
df_col['var1']='` FLOAT NULL,'
df_col['var']='`'
df_col = pd.DataFrame.reindex(df_col,columns = ['var', 'cols','var1'])
df_col['var1'] = df_col['var1'].astype(str)
df_col['var'] = df_col['var'].astype(str)
df_col['cols'] = df_col['cols'].astype(str)
df_col = df_col.iloc[1:]

df_col["sql"] = df_col['var'].astype(str) +df_col['cols']+ df_col['var1']

df_col

,var,cols,var1,sql
1,`,0088AA,"` FLOAT NULL,","`0088AA` FLOAT NULL,"
2,`,0085AA,"` FLOAT NULL,","`0085AA` FLOAT NULL,"
3,`,0076AA,"` FLOAT NULL,","`0076AA` FLOAT NULL,"
4,`,0064AA,"` FLOAT NULL,","`0064AA` FLOAT NULL,"
5,`,0049AA,"` FLOAT NULL,","`0049AA` FLOAT NULL,"
6,`,0044AA,"` FLOAT NULL,","`0044AA` FLOAT NULL,"
7,`,0007AA,"` FLOAT NULL,","`0007AA` FLOAT NULL,"


In [131]:
df_col.to_csv(r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Update Files\RH_update_cols.csv', index = False)

In [38]:
df_col = pd.DataFrame(df.columns)
df_col['var1']=' set'
df_col['var']='update '
df_col['var3']='rh_all '
df_col['var2']='= null where'
df_col['var4']='=0;'
df_col = pd.DataFrame.reindex(df_col,columns = ['var', 'var3','var1', 0,'var2',0,'var4'])
df_col['var1'] = df_col['var1'].astype(str)
df_col['var'] = df_col['var'].astype(str)
df_col[0] = df_col[0].astype(str)
df_col = df_col.iloc[1:]

#df_col["sql"] = df_col['var'].astype(str) +df_col['cols']+ df_col['var1']

df_col

,var,var3,var1,0,var2,0,var4
1,update,rh_all,set,0088AA,= null where,0088AA,=0;
2,update,rh_all,set,0085AA,= null where,0085AA,=0;
3,update,rh_all,set,0076AA,= null where,0076AA,=0;
4,update,rh_all,set,0064AA,= null where,0064AA,=0;
5,update,rh_all,set,0049AA,= null where,0049AA,=0;
6,update,rh_all,set,0044AA,= null where,0044AA,=0;
7,update,rh_all,set,0007AA,= null where,0007AA,=0;


In [169]:
df_col.to_csv(r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Update Files\null_code_pilot.csv', index = False)

In [40]:
stats = df.describe()
stats

,084AA,0088AA,0085AA,0076AA,0064AA,0049AA,0044AA,0007AA
count,1734.000000,1729.000000,1731.000000,1731.000000,1652.000000,1731.000000,1731.000000,1732.000000
mean,84.360321,77.259881,82.531449,85.227124,81.449925,76.657901,78.742255,81.264790
std,7.174725,11.299652,6.469811,6.704472,8.086880,11.146447,9.420363,6.945835
min,64.354000,47.199000,63.541000,63.513000,50.767000,47.736000,53.906000,58.754000
25%,79.944000,70.142000,79.300000,82.369000,76.563000,69.116000,72.839000,77.014000
50%,86.632000,81.683000,83.923000,87.105000,83.059000,80.651000,81.967000,83.819500
75%,89.914000,86.231000,87.340000,90.112000,88.434000,85.915500,86.441000,86.589000
max,94.468000,91.779000,94.725000,95.928000,93.796000,91.249000,90.533000,90.292000


null values

In [46]:
df.isnull().sum()

084AA      4
0088AA     9
0085AA     7
0076AA     7
0064AA    86
0049AA     7
0044AA     7
0007AA     6
dtype: int64